In [7]:
import nibabel as nl
import numpy as np
import pandas as pd
import os, shutil
import matplotlib.pyplot as plt

In [8]:
df = pd.read_csv('final.csv')
df.head()

,Subject_ID,sws_rw,sws_rs,sws_ms,sws_fa,sws_gb,sws_total,zDCwei_17Networks_RH_TempPar_10,zDCwei_17Networks_RH_TempPar_9,zDCwei_17Networks_RH_TempPar_8,...,zReHo_17Networks_RH_DorsAttnA_SPL_3,zReHo_17Networks_RH_DorsAttnA_SPL_2,zReHo_17Networks_RH_DorsAttnA_SPL_1,zReHo_17Networks_RH_DorsAttnA_ParOcc_3,zReHo_17Networks_RH_DorsAttnA_ParOcc_2,zReHo_17Networks_RH_DorsAttnA_ParOcc_1,zReHo_17Networks_RH_DorsAttnA_TempOcc_3,zReHo_17Networks_RH_DorsAttnA_TempOcc_2,zReHo_17Networks_RH_DorsAttnA_TempOcc_1,zReHo_17Networks_RH_SomMotB_Cent_3
0,sub-0015,3.4,1.666667,1.833333,1.000000,1.4,1.513333,0.253102,0.000837,0.268018,...,0.632937,0.484317,0.878408,0.776844,0.120077,0.906160,1.010050,0.325154,0.070342,0.635567
1,sub-0085,2.8,2.166667,3.500000,2.166667,2.8,2.560000,0.497937,0.553454,0.635488,...,1.272547,0.551014,1.482868,1.110642,0.290340,0.240267,0.374478,-0.029872,-0.361664,0.516531
2,sub-0134,3.8,2.000000,2.166667,2.166667,3.4,2.346667,0.896547,0.253496,0.506752,...,0.510567,0.323151,0.741446,0.395507,0.640683,0.381234,0.707233,0.149791,-0.279489,0.050742
3,sub-0152,4.0,1.666667,1.500000,2.500000,3.0,2.066667,-0.154433,-0.033360,0.157313,...,-0.357383,-0.159408,0.082269,0.183071,-0.415082,-0.233840,-0.117080,0.584565,0.458543,0.246564
4,sub-0260,3.8,2.500000,3.600000,3.000000,4.0,3.120000,-0.266587,0.072713,0.212338,...,0.387290,0.171799,0.892404,0.532177,0.216433,0.554115,0.634269,0.389356,-0.211930,0.565683


In [5]:
df.shape

(118, 1024)

## compeleting df with schaefer 400

### list of ALFFs

In [6]:
# opening the text file
lis = []
with open('allzALFF_meants_400.txt','r') as ALFFout:
   
    # reading each line    
    for line in ALFFout:
   
        # reading each word        
        for ALFF in line.split():
            lis.append(float(ALFF))
            # displaying the words           
            #print(ALFF) 

### list of region names in Schaefer Atlas (400 parc)

In [8]:
token = open('Schaefer2018_400Parcels_17Networks_order.txt','r')
linestoken=token.readlines()
tokens_column_number = 1
parname=[]
for x in linestoken:
    parname.append(x.split()[tokens_column_number])
token.close()
print(parname)

['17Networks_LH_VisCent_ExStr_1', '17Networks_LH_VisCent_ExStr_2', '17Networks_LH_VisCent_ExStr_3', '17Networks_LH_VisCent_ExStr_4', '17Networks_LH_VisCent_ExStr_5', '17Networks_LH_VisCent_ExStr_6', '17Networks_LH_VisCent_Striate_1', '17Networks_LH_VisCent_ExStr_7', '17Networks_LH_VisCent_ExStr_8', '17Networks_LH_VisCent_ExStr_9', '17Networks_LH_VisCent_ExStr_10', '17Networks_LH_VisCent_ExStr_11', '17Networks_LH_VisPeri_ExStrInf_1', '17Networks_LH_VisPeri_ExStrInf_2', '17Networks_LH_VisPeri_ExStrInf_3', '17Networks_LH_VisPeri_ExStrInf_4', '17Networks_LH_VisPeri_ExStrInf_5', '17Networks_LH_VisPeri_StriCal_1', '17Networks_LH_VisPeri_StriCal_2', '17Networks_LH_VisPeri_ExStrSup_1', '17Networks_LH_VisPeri_ExStrSup_2', '17Networks_LH_VisPeri_ExStrSup_3', '17Networks_LH_VisPeri_ExStrSup_4', '17Networks_LH_VisPeri_ExStrSup_5', '17Networks_LH_SomMotA_1', '17Networks_LH_SomMotA_2', '17Networks_LH_SomMotA_3', '17Networks_LH_SomMotA_4', '17Networks_LH_SomMotA_5', '17Networks_LH_SomMotA_6', '17Netw

### merging regions and their values with dataframe

In [9]:
test = df.set_index('Subject_ID')
inds = test.index

In [12]:
df = pd.DataFrame()
for j, ind in enumerate(inds):
    sub = test.loc[[ind]]
    for i, pars in enumerate(parname):
        col_name = 'zALFF_'+str(pars)
        if col_name not in sub:
            sub.insert(2, col_name, np.nan, True)
        sub[col_name] = lis[i+j*400]
    df = pd.concat([df, sub])

In [13]:
df.shape

(118, 1423)

In [15]:
df.to_csv('df with Schaefer400_DCwei-DCbin-ReHo-ALFF_and_BN DCwei-DCbin.csv')

In [13]:
df4 = df[['Subject_ID','sws_ms','sws_fa', 'sws_gb', 'sws_total']]

In [14]:
df4.to_csv("df4.csv")